In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 6.2 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [5]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [7]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2025-09-07 19:12:40,648] A new study created in memory with name: no-name-508f5b4f-c7b7-4f15-82ac-824bae4c7930
[I 2025-09-07 19:12:41,216] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 113, 'max_depth': 13}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-07 19:12:41,492] Trial 1 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 67, 'max_depth': 7}. Best is trial 1 with value: 0.7709497206703911.
[I 2025-09-07 19:12:42,077] Trial 2 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 146, 'max_depth': 18}. Best is trial 1 with value: 0.7709497206703911.
[I 2025-09-07 19:12:42,304] Trial 3 finished with value: 0.7802607076350093 and parameters: {'n_estimators': 54, 'max_depth': 7}. Best is trial 3 with value: 0.7802607076350093.
[I 2025-09-07 19:12:42,932] Trial 4 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 153, 'max_depth': 11}. Best is trial 3 with value: 0.78026070

In [8]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7821229050279329
Best hyperparameters: {'n_estimators': 117, 'max_depth': 15}


In [9]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


optuna visualisation

In [10]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [11]:
# 1. Optimization History
plot_optimization_history(study).show()

In [12]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [13]:
# 3. Slice Plot
plot_slice(study).show()

In [14]:
# 4. Contour Plot
plot_contour(study).show()

In [15]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [16]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [17]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [18]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-09-07 19:51:45,342] A new study created in memory with name: no-name-97cb2f86-ceea-426f-9093-83238ebfd17e
[I 2025-09-07 19:51:45,366] Trial 0 finished with value: 0.7337057728119181 and parameters: {'classifier': 'SVM', 'C': 2.00632603965562, 'kernel': 'poly', 'gamma': 'scale'}. Best is trial 0 with value: 0.7337057728119181.
[I 2025-09-07 19:51:46,398] Trial 1 finished with value: 0.7597765363128491 and parameters: {'classifier': 'RandomForest', 'n_estimators': 283, 'max_depth': 10, 'min_samples_split': 6, 'min_samples_leaf': 7, 'bootstrap': True}. Best is trial 1 with value: 0.7597765363128491.
[I 2025-09-07 19:51:46,458] Trial 2 finished with value: 0.7858472998137801 and parameters: {'classifier': 'SVM', 'C': 14.40154655161757, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 2 with value: 0.7858472998137801.
[I 2025-09-07 19:51:46,517] Trial 3 finished with value: 0.7858472998137801 and parameters: {'classifier': 'SVM', 'C': 13.933250773045279, 'kernel': 'linear', 'gamm

In [19]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.13027833369060426, 'kernel': 'linear', 'gamma': 'auto'}
Best trial accuracy: 0.7895716945996275


In [20]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.733706,2025-09-07 19:51:45.344307,2025-09-07 19:51:45.366208,0 days 00:00:00.021901,2.006326,NaN,SVM,scale,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
1,1,0.759777,2025-09-07 19:51:45.367272,2025-09-07 19:51:46.398026,0 days 00:00:01.030754,NaN,True,RandomForest,NaN,NaN,NaN,10.0,7.0,6.0,283.0,COMPLETE
2,2,0.785847,2025-09-07 19:51:46.398981,2025-09-07 19:51:46.458848,0 days 00:00:00.059867,14.401547,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
3,3,0.785847,2025-09-07 19:51:46.459837,2025-09-07 19:51:46.517664,0 days 00:00:00.057827,13.933251,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
4,4,0.776536,2025-09-07 19:51:46.518767,2025-09-07 19:51:46.774197,0 days 00:00:00.255430,NaN,False,RandomForest,NaN,NaN,NaN,19.0,8.0,10.0,79.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.785847,2025-09-07 19:52:14.796501,2025-09-07 19:52:14.819534,0 days 00:00:00.023033,0.221676,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.756052,2025-09-07 19:52:14.820716,2025-09-07 19:52:14.851130,0 days 00:00:00.030414,0.167140,NaN,SVM,scale,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.722533,2025-09-07 19:52:14.852308,2025-09-07 19:52:14.882792,0 days 00:00:00.030484,0.254485,NaN,SVM,scale,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.763501,2025-09-07 19:52:14.883757,2025-09-07 19:52:16.067284,0 days 00:00:01.183527,NaN,NaN,GradientBoosting,NaN,NaN,0.011174,20.0,9.0,5.0,144.0,COMPLETE


In [21]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,80
RandomForest,10
GradientBoosting,10


In [22]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.739851
RandomForest,0.764804
SVM,0.773068


In [23]:
# 1. Optimization History
plot_optimization_history(study).show()

In [24]:
# 3. Slice Plot
plot_slice(study).show()

In [25]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()